<a href="https://colab.research.google.com/github/Raunak-23/EvalAI/blob/main/ml/ocr/ocr_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install opencv-python

In [17]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import os
import random
import time
from os import walk
import cv2
import matplotlib.pyplot as plt
import numpy as np
import torch
from skimage.feature import local_binary_pattern
from skimage.measure import find_contours
from skimage.morphology import binary_dilation
from sklearn.svm import SVC
from torch import nn, optim
from torch.utils.data import TensorDataset
import torch.nn.functional as F

In [3]:
# Parameters and constants
AVAILABLE_WRITERS = 672
RESULTS_FILE = 'results.txt'
TIME_FILE = 'time.txt'
OVERLAPPING_METHOD = 0
LINES_METHOD = 1
SUPPORT_VECTOR_CLASSIFIER = 0
NEURAL_NETWORK_CLASSIFIER = 1
HISTOGRAM_BINS = 256
NN_LEARNING_RATE = 0.003
NN_WEIGHT_DECAY = 0.01
NN_DROPOUT = 0.25
NN_EPOCHS = 200
NN_BATCH_SIZE = 16
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [4]:
def show_images(images, titles=None):
    n_ims = len(images)
    if titles is None:
        titles = ['(%d)' % i for i in range(1, n_ims + 1)]
    fig = plt.figure()
    n = 1
    for image, title in zip(images, titles):
        a = fig.add_subplot(1, n_ims, n)
        if image.ndim == 2:
            plt.gray()
        plt.imshow(image)
        a.set_title(title)
        n += 1
    fig.set_size_inches(np.array(fig.get_size_inches()) * n_ims)
    plt.show()

In [5]:
def preprocess_image(img, feature_extraction_method=OVERLAPPING_METHOD):
    if feature_extraction_method == OVERLAPPING_METHOD:
        img_copy = img.copy()
        if len(img.shape) > 2:
            img_copy = cv2.cvtColor(img_copy, cv2.COLOR_BGR2GRAY)
        img_copy = cv2.medianBlur(img_copy, 5)
        img_copy = cv2.threshold(img_copy, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]
        min_vertical, max_vertical = get_corpus_boundaries(img_copy)
        img_copy = img_copy[min_vertical:max_vertical]
        return img_copy

    if feature_extraction_method == LINES_METHOD:
        img_copy = img.copy()
        if len(img.shape) > 2:
            grayscale_img = cv2.cvtColor(img_copy, cv2.COLOR_BGR2GRAY)
        else:
            grayscale_img = img.copy()
        img_copy = cv2.threshold(grayscale_img, 127, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]
        min_vertical, max_vertical = get_corpus_boundaries(img_copy)
        img_copy = img_copy[min_vertical:max_vertical]
        grayscale_img = grayscale_img[min_vertical:max_vertical]
        filter_kernel = np.array([[-1, -1, -1], [-1, 9, -1], [-1, -1, -1]])
        img_copy_sharpened = cv2.filter2D(img_copy, -1, filter_kernel)
        return img_copy_sharpened, grayscale_img

In [6]:
def get_corpus_boundaries(img):
    crop = []
    horizontal_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (100, 1))
    detect_horizontal = cv2.morphologyEx(img, cv2.MORPH_OPEN, horizontal_kernel, iterations=2)
    contours = cv2.findContours(detect_horizontal, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contours = contours[0] if len(contours) == 2 else contours[1]
    prev = -1
    for i, c in enumerate(contours):
        if np.abs(prev - int(c[0][0][1])) > 800 or prev == -1:
            crop.append(int(c[0][0][1]))
            prev = int(c[0][0][1])
    crop.sort()
    max_vertical = crop[1] - 20
    min_vertical = crop[0] + 20
    return min_vertical, max_vertical

In [7]:
def segment_image(img, num, grayscale_img=None):
    if grayscale_img is not None:
        grayscale_images = []
        img_copy = np.copy(img)
        kernel = np.ones((1, num))
        img_copy = binary_dilation(img_copy, kernel)
        bounding_boxes = find_contours(img_copy, 0.8)
        for box in bounding_boxes:
            x_min = int(np.min(box[:, 1]))
            x_max = int(np.max(box[:, 1]))
            y_min = int(np.min(box[:, 0]))
            y_max = int(np.max(box[:, 0]))
            if (y_max - y_min) > 50 and (x_max - x_min) > 50:
                grayscale_images.append(grayscale_img[y_min:y_max, x_min:x_max])
        return grayscale_images
    images = []
    img_copy = np.copy(img)
    kernel = np.ones((1, num))
    img_copy = binary_dilation(img_copy, kernel)
    bounding_boxes = find_contours(img_copy, 0.8)
    for box in bounding_boxes:
        x_min = int(np.min(box[:, 1]))
        x_max = int(np.max(box[:, 1]))
        y_min = int(np.min(box[:, 0]))
        y_max = int(np.max(box[:, 0]))
        if (y_max - y_min) > 10 and (x_max - x_min) > 10:
            images.append(img[y_min:y_max, x_min:x_max])
    return images

In [8]:
def overlap_words(words, avg_height):
    overlapped_img = np.zeros((3600, 320))
    index_i = 0
    index_j = 0
    max_height = 0
    for word in words:
        if word.shape[1] + index_j > overlapped_img.shape[1]:
            max_height = 0
            index_j = 0
            index_i += int(avg_height // 2)
        if word.shape[1] < overlapped_img.shape[1] and word.shape[0] < overlapped_img.shape[0]:
            indices = np.copy(overlapped_img[index_i:index_i + word.shape[0], index_j:index_j + word.shape[1]])
            indices = np.maximum(indices, word)
            overlapped_img[index_i:index_i + word.shape[0], index_j:index_j + word.shape[1]] = indices
            index_j += word.shape[1]
            if max_height < word.shape[0]:
                max_height = word.shape[0]
    overlapped_img = overlapped_img[:index_i + int(avg_height // 2), :]
    return overlapped_img

In [9]:
def get_textures(image):
    index_i = 0
    index_j = 0
    texture_size = 100
    textures = []
    while index_i + texture_size < image.shape[0]:
        if index_j + texture_size > image.shape[1]:
            index_j = 0
            index_i += texture_size
        textures.append(np.copy(image[index_i: index_i + texture_size, index_j: index_j + texture_size]))
        index_j += texture_size
    return textures

In [10]:
def model_generator(features, labels, feature_extraction_method=OVERLAPPING_METHOD,
                    classifier_type=SUPPORT_VECTOR_CLASSIFIER):
    histograms = []

    if feature_extraction_method == OVERLAPPING_METHOD:
        for texture_array in features:
            for texture in texture_array:
                lbp = local_binary_pattern(texture, 8, 3, 'default')
                histogram, _ = np.histogram(lbp, density=False, bins=HISTOGRAM_BINS, range=(0, HISTOGRAM_BINS))
                histograms.append(histogram)

    elif feature_extraction_method == LINES_METHOD:
        for line in features:
            lbp = local_binary_pattern(line, 8, 3, 'default')
            histogram, _ = np.histogram(lbp, density=False, bins=HISTOGRAM_BINS, range=(0, HISTOGRAM_BINS))
            histograms.append(histogram)

    if classifier_type == SUPPORT_VECTOR_CLASSIFIER:
        model = SVC(kernel='linear')
        model.fit(histograms, labels)
        return model

    if classifier_type == NEURAL_NETWORK_CLASSIFIER:
        model = nn.Sequential(nn.Linear(HISTOGRAM_BINS, 128),
                              nn.ReLU(),
                              nn.Dropout(p=NN_DROPOUT),
                              nn.Linear(128, 64),
                              nn.ReLU(),
                              nn.Dropout(p=NN_DROPOUT),
                              nn.Linear(64, 3))
        model.to(DEVICE)
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adamax(model.parameters(), lr=NN_LEARNING_RATE, weight_decay=NN_WEIGHT_DECAY)
        inputs = torch.Tensor(histograms)
        labels = torch.tensor(labels, dtype=torch.long) - 1
        dataset = TensorDataset(inputs, labels)
        train_loader = torch.utils.data.DataLoader(dataset, batch_size=NN_BATCH_SIZE, shuffle=True)
        for epoch in range(NN_EPOCHS):
            for inputs, labels in train_loader:
                inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
                output = model(inputs)
                loss = criterion(output, labels)
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
        return model

In [11]:
def predict(model, test_image, feature_extraction_method=OVERLAPPING_METHOD, classifier_type=SUPPORT_VECTOR_CLASSIFIER):
    if feature_extraction_method == OVERLAPPING_METHOD:
        img = preprocess_image(test_image)
        words = segment_image(img, 3)
        avg_height = 0
        for word in words:
            avg_height += word.shape[0] / len(words)
        overlapped_img = overlap_words(words, avg_height)
        textures = get_textures(overlapped_img)
        prediction = np.zeros(4)
        for texture in textures:
            lbp = local_binary_pattern(texture, 8, 3, 'default')
            histogram, _ = np.histogram(lbp, density=False, bins=HISTOGRAM_BINS, range=(0, HISTOGRAM_BINS))
            if classifier_type == SUPPORT_VECTOR_CLASSIFIER:
                prediction[model.predict([histogram])] += 1
            if classifier_type == NEURAL_NETWORK_CLASSIFIER:
                with torch.no_grad():
                    model.eval()
                    histogram = torch.Tensor(histogram)
                    probabilities = F.softmax(model.forward(histogram), dim=0)
                    _, top_class = probabilities.topk(1)
                    prediction[top_class + 1] += 1
        return np.argmax(prediction)

    if feature_extraction_method == LINES_METHOD:
        img, grayscale_img = preprocess_image(test_image, feature_extraction_method)
        grayscale_lines = segment_image(img, 100, grayscale_img)
        prediction = np.zeros(4)
        for line in grayscale_lines:
            lbp = local_binary_pattern(line, 8, 3, 'default')
            histogram, _ = np.histogram(lbp, density=False, bins=HISTOGRAM_BINS, range=(0, HISTOGRAM_BINS))
            if classifier_type == SUPPORT_VECTOR_CLASSIFIER:
                prediction[model.predict([histogram])] += 1
            if classifier_type == NEURAL_NETWORK_CLASSIFIER:
                with torch.no_grad():
                    model.eval()
                    histogram = torch.Tensor(histogram)
                    probabilities = F.softmax(model.forward(histogram), dim=0)
                    _, top_class = probabilities.topk(1)
                    prediction[top_class + 1] += 1
        return np.argmax(prediction)

In [12]:
def read_random_images(root):
    images = []
    labels = []
    test_images = []
    test_labels = []
    for i in range(3):
        found_images = False
        while not found_images:
            images_path = root
            random_writer = random.randrange(AVAILABLE_WRITERS)
            if random_writer < 10:
                random_writer = "00" + str(random_writer)
            elif random_writer < 100:
                random_writer = "0" + str(random_writer)
            images_path = os.path.join(images_path, str(random_writer))
            if not os.path.isdir(images_path):
                continue
            _, _, filenames = next(walk(images_path))
            if len(filenames) <= 2 and i == 2 and len(test_images) == 0:
                continue
            if len(filenames) >= 2:
                found_images = True
                chosen_filenames = []
                for j in range(2):
                    random_filename = random.choice(filenames)
                    while random_filename in chosen_filenames:
                        random_filename = random.choice(filenames)
                    chosen_filenames.append(random_filename)
                    images.append(cv2.imread(os.path.join(images_path, random_filename)))
                    labels.append(i + 1)
                if len(filenames) >= 3:
                    random_filename = random.choice(filenames)
                    while random_filename in chosen_filenames:
                        random_filename = random.choice(filenames)
                    chosen_filenames.append(random_filename)
                    test_images.append(cv2.imread(os.path.join(images_path, random_filename)))
                    test_labels.append(i + 1)
    test_choice = random.randint(0, len(test_images) - 1)
    test_image = test_images[test_choice]
    test_label = test_labels[test_choice]
    return images, labels, test_image, test_label

In [13]:
def extract_features(images, labels, feature_extraction_method=OVERLAPPING_METHOD):
    if feature_extraction_method == LINES_METHOD:
        lines_labels = []
        lines = []
        for image, label in zip(images, labels):
            image, grayscale_image = preprocess_image(image, feature_extraction_method)
            grayscale_lines = segment_image(image, 100, grayscale_image)
            for line in grayscale_lines:
                lines.append(line)
                lines_labels.append(label)
        return lines, lines_labels

    if feature_extraction_method == OVERLAPPING_METHOD:
        textures = []
        textures_labels = []
        for image, label in zip(images, labels):
            image = preprocess_image(image)
            words = segment_image(image, 3)
            avg_height = 0
            for word in words:
                avg_height += word.shape[0] / len(words)
            overlapped_img = overlap_words(words, avg_height)
            new_textures = get_textures(overlapped_img)
            textures.append(new_textures)
            for j in range(len(new_textures)):
                textures_labels.append(label)
        return textures, textures_labels

In [14]:
# !pip install kagglehub

In [15]:
# import kagglehub

# # Download latest version
# path = kagglehub.dataset_download("naderabdalghani/iam-handwritten-forms-dataset")

# print("Path to dataset files:", path)

In [19]:
# ==========  PARAMETERS  ==========
OVERLAPPING_METHOD   = 0
SUPPORT_VECTOR_CLASSIFIER = 0
AVAILABLE_WRITERS    = 672          # IAM has 657 writers – we take a safe upper bound
epochs               = 10           # start small – increase later
root                 = "/content/drive/MyDrive/data/iam_words/words"   # <— change if different
# ==================================

correct_predictions   = 0
total_execution_time  = 0

def collect_image_paths(root_dir, writers_needed=3, files_per_writer=2):
    """
    Returns  ( [(path, label), …], test_path, test_label )
    labels are 1,2,3  (writer index)
    """
    import random, os, glob
    writers = [d for d in os.listdir(root_dir)
               if os.path.isdir(os.path.join(root_dir, d))]
    random.shuffle(writers)
    out, labels = [], []
    for w_idx, writer in enumerate(writers[:writers_needed]):
        writer_dir = os.path.join(root_dir, writer)
        pages = [d for d in os.listdir(writer_dir)
                 if os.path.isdir(os.path.join(writer_dir, d))]
        if not pages:
            continue
        # pick one page for this writer
        page = random.choice(pages)
        page_dir = os.path.join(writer_dir, page)
        pngs = glob.glob(os.path.join(page_dir, "*.png"))
        if len(pngs) < files_per_writer + 1:        # need 2 train + 1 test
            continue
        sample = random.sample(pngs, files_per_writer + 1)
        train = sample[:-1]
        test  = sample[-1]
        for p in train:
            out.append(p)
            labels.append(w_idx + 1)                 # label 1,2,3
    if len(out) < writers_needed * files_per_writer:
        raise RuntimeError("Not enough images – check folder structure")
    return out, labels, test, w_idx + 1


for epoch in range(epochs):
    try:
        train_paths, train_labels, test_path, test_label = collect_image_paths(root)
    except RuntimeError as e:
        print(e)
        break

    images = [cv2.imread(p) for p in train_paths]
    start = time.time()

    features, features_labels = extract_features(images, train_labels,
                                                 feature_extraction_method=OVERLAPPING_METHOD)
    model = model_generator(features, features_labels,
                            feature_extraction_method=OVERLAPPING_METHOD,
                            classifier_type=SUPPORT_VECTOR_CLASSIFIER)

    test_img = cv2.imread(test_path)
    pred = predict(model, test_img,
                   feature_extraction_method=OVERLAPPING_METHOD,
                   classifier_type=SUPPORT_VECTOR_CLASSIFIER)

    elapsed = time.time() - start
    total_execution_time += elapsed
    if pred == test_label:
        correct_predictions += 1

    print(f"Epoch {epoch+1:03d} | "
          f"time {elapsed:5.2f}s | "
          f"accuracy {100*correct_predictions/(epoch+1):5.2f}%")

print("\nFinal accuracy : {:.2f} %  ({} epochs)".format(
      100 * correct_predictions / epochs, epochs))

Not enough images – check folder structure

Final accuracy : 0.00 %  (10 epochs)


In [22]:
#!/usr/bin/env python3
"""
flatten_iam_words.py
Flatten the IAM-word nested folder structure.
Before:  iam_words/words/a01/a01-000u/a01-000u-00-00.png
After:   iam_words/words_flat/a01-000u-00-00.png
"""

import os
import shutil
from pathlib import Path
from tqdm import tqdm   # pip install tqdm  (optional, nicer progress bar)

# ------------- CONFIG --------------
SRC = Path("/content/drive/MyDrive/data/iam_words/words")   # change if necessary
DST = Path("/content/drive/MyDrive/data/iam_words/words_flat")                        # same parent dir
# -----------------------------------

DST.mkdir(exist_ok=True)

pngs = list(SRC.rglob("*.png"))
print(f"Found {len(pngs)} PNG files.")

for file in tqdm(pngs, desc="Copying"):
    target = DST / file.name
    # handle extremely unlikely name collision
    counter = 1
    stem = target.stem
    suffix = target.suffix
    while target.exists():
        target = DST / f"{stem}_{counter}{suffix}"
        counter += 1
    shutil.copy2(file, target)

print("Done – flat folder:", DST.resolve())

Found 63766 PNG files.


Copying:  15%|█▌        | 9632/63766 [46:26<4:21:02,  3.46it/s]


KeyboardInterrupt: 